In [1]:
import os

#initialize the params
target="\\Cali\\4K\\CAM_3D"

#change directory for saving images
if os.getcwd().endswith(target)==False:
    PATH=os.getcwd() + target
    os.chdir(PATH)

print(os.getcwd())

c:\Users\User\Desktop\Dev\tabletennis_robot\Cali\4K\CAM_3D


In [2]:
import cv2

camera_num=0
cap=cv2.VideoCapture(camera_num)

#change camera frame to 1280 X 720
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

#print changed frame
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print('changed resolution width {} height {}'.format(width, height))

changed resolution width 1280.0 height 720.0


In [6]:
#capture the checkerboard
import cv2
import os

#initialize the params
pic_num=0

camera_num=0
cap=cv2.VideoCapture(camera_num)

#change camera frame to 1280 X 720
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)

if cap.isOpened():
    #print the information of current camera
    print("frame type: ", cap.get(cv2.CAP_PROP_FRAME_TYPE))
    print("frame height: ", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print("frame width: ", cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    #capture the checkerboard images
    while True:
        ret, img=cap.read()
        if ret:
            cv2.imshow('camera', img)
            # command=input("Waiting for yout keyboard press: c for capture, q for quit")
            command=cv2.waitKey(1)
            if command==ord("c"):
                    cv2.imwrite(f"checkerboard{pic_num}_cam{camera_num}.jpg", img)
                    pic_num+=1
            elif command==ord("q"):
                break
        else:
            print("no frame!")
            break
    cap.release()
else:
    print("cannot open camera")
cap.release()
cv2.destroyAllWindows()

frame type:  -1.0
frame height:  720.0
frame width:  1280.0


KeyboardInterrupt: 

In [3]:
#rvec, tvec for 3D mapping
import cv2
import numpy as np
import os

#define the board's dimension
CHECKERBOARD=(5, 7)
criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100000, 0.000000001)

#generate vector of 3D points
objpoints=[]

#generate vector of 2D points
imgpoints=[]

#define world matrix
objp=np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0, :, :2]=np.mgrid[0: CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape=None

fname="cam0_ex.jpg"
img=cv2.imread(fname)

#invert into gray scale
gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#find corners of the checkerboard
ret, corners=cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
print("cv2 corner detection result: ", ret)

#if the desired number of corners is found
if ret==True:
    objpoints.append(objp)
    
    #fine tunning the matrix based on given 2D points
    corners2=cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria) 
    imgpoints.append(corners2)
    # print("corners2: ", corners2)

    #illustrate the corners
    img=cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)

# elif ret==False:
#     os.remove(fname)
    
cv2.imshow('cali result', img)
cv2.waitKey(0)



cv2.destroyAllWindows()
h, w=img.shape[:2]
print("h, w: ", h, w)

#check the results
print("objpoints: ", objpoints)

#with the true 3D points and given 2D points, do calibration
ret, mtx, dist, rvecs, tvecs=cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)



cv2 corner detection result:  True
h, w:  720 1280
objpoints:  [array([[[0., 0., 0.],
        [1., 0., 0.],
        [2., 0., 0.],
        [3., 0., 0.],
        [4., 0., 0.],
        [0., 1., 0.],
        [1., 1., 0.],
        [2., 1., 0.],
        [3., 1., 0.],
        [4., 1., 0.],
        [0., 2., 0.],
        [1., 2., 0.],
        [2., 2., 0.],
        [3., 2., 0.],
        [4., 2., 0.],
        [0., 3., 0.],
        [1., 3., 0.],
        [2., 3., 0.],
        [3., 3., 0.],
        [4., 3., 0.],
        [0., 4., 0.],
        [1., 4., 0.],
        [2., 4., 0.],
        [3., 4., 0.],
        [4., 4., 0.],
        [0., 5., 0.],
        [1., 5., 0.],
        [2., 5., 0.],
        [3., 5., 0.],
        [4., 5., 0.],
        [0., 6., 0.],
        [1., 6., 0.],
        [2., 6., 0.],
        [3., 6., 0.],
        [4., 6., 0.]]], dtype=float32)]


In [12]:
#with checkerboard images, do calibration
import cv2
import numpy as np
import os
import glob


#define the board's dimension
CHECKERBOARD=(7, 5)
criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

#generate vector of 3D points
objpoints=[]

#generate vector of 2D points
imgpoints=[]

#define world matrix
objp=np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0, :, :2]=np.mgrid[0: CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape=None

#extract the path of checkerboard images
# target='./Cali/4K/CAM1/*.jpg'

images=glob.glob(os.getcwd()+"/*.jpg")
print(images)


for fname in images:
    img=cv2.imread(fname)

    #invert into gray scale
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #find corners of the checkerboard
    ret, corners=cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    print("cv2 corner detection result: ", ret)
    
    #if the desired number of corners is found
    if ret==True:
        objpoints.append(objp)
        
        #fine tunning the matrix based on given 2D points
        corners2=cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria) 
        imgpoints.append(corners2)
        # print("corners2: ", corners2)

        #illustrate the corners
        img=cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)

    elif ret==False:
        os.remove(fname)
        
    cv2.imshow('cali result', img)
    cv2.waitKey(0)



cv2.destroyAllWindows()
h, w=img.shape[:2]
print("h, w: ", h, w)

#check the results
print("objpoints: ", objpoints)

#with the true 3D points and given 2D points, do calibration
ret, mtx, dist, rvecs, tvecs=cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)



['c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM_3D\\cam0_ex.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM_3D\\cam1_ex.jpg']
cv2 corner detection result:  True
cv2 corner detection result:  True
h, w:  720 1280
objpoints:  [array([[[0., 0., 0.],
        [1., 0., 0.],
        [2., 0., 0.],
        [3., 0., 0.],
        [4., 0., 0.],
        [5., 0., 0.],
        [6., 0., 0.],
        [0., 1., 0.],
        [1., 1., 0.],
        [2., 1., 0.],
        [3., 1., 0.],
        [4., 1., 0.],
        [5., 1., 0.],
        [6., 1., 0.],
        [0., 2., 0.],
        [1., 2., 0.],
        [2., 2., 0.],
        [3., 2., 0.],
        [4., 2., 0.],
        [5., 2., 0.],
        [6., 2., 0.],
        [0., 3., 0.],
        [1., 3., 0.],
        [2., 3., 0.],
        [3., 3., 0.],
        [4., 3., 0.],
        [5., 3., 0.],
        [6., 3., 0.],
        [0., 4., 0.],
        [1., 4., 0.],
        [2., 4., 0.],
        [3., 4., 0.],
        [4., 4., 0.],
        

In [4]:
print("camera matrix: \n")
print(mtx)

camera matrix: 

[[791.73211828   0.         664.95863988]
 [  0.         842.57596176 356.44151064]
 [  0.           0.           1.        ]]


In [5]:
print("dist: \n")
print(dist)

dist: 

[[ 0.65431131  0.49894282 -0.14079931  0.00907266 -5.05880803]]


In [4]:
print("rvecs: \n")
print(rvecs)

rvecs: 

(array([[-0.24140184],
       [-0.03802079],
       [ 0.0033663 ]]),)


In [6]:
print("tvecs: \n")
print(tvecs)

tvecs: 

(array([[-2.70205959],
       [11.48478121],
       [25.77444696]]),)


In [7]:
#human error: current directory forgetting
print(os.getcwd())
target='./Cali/4K/CAM2'
images=glob.glob(target)
print(images)
for fname in images:
    print(fname)

c:\Users\User\Desktop\Dev\tabletennis_robot\Cali\4K\CAM2
[]


In [39]:
img=cv2.imread('./checkerboard0_cam1.jpg')
h, w=img.shape[:2]

#obtain newcameramtx, roi for calibration
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

In [40]:
#show the calibrated image
mapx, mapy=cv2.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w, h), 5)
dst=cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)

x, y, w, h=roi
dst=dst[y:y+h, x:x+w]
cv2.imwrite('cali_remap0_cam1.png', dst)


True